# ML.Net - ExpressionTransform

In [1]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML" 

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.ML version 1.5.0

## Using C# Class

In [2]:
using System;
using Microsoft.ML;
using Microsoft.ML.Data;

## Declare data-classes for input data and predictions

In [3]:
public class SalaryInput
{
    public float YearsExperience;

    public bool IsManager;

    public string Title;

    public int NumberOfTeamsManaged;
}

public class ExpressionOutput
{
    public int TeamsManagedOutput { get; set; }
    public float SquareRootOutput { get; set; }
    public string ToLowerOutput { get; set; }
}

## Evaluate

In [4]:
var context = new MLContext();

var inputData = new List<SalaryInput>
{
    new SalaryInput { IsManager = false, YearsExperience = 1f, Title = "Developer" },
    new SalaryInput { IsManager = true, YearsExperience = 9f, Title = "Director", NumberOfTeamsManaged = 2 },
    new SalaryInput { IsManager = false, YearsExperience = 4f, Title = "Analyst" }
};

var data = context.Data.LoadFromEnumerable(inputData);

var expressions = context.Transforms.Expression("SquareRootOutput", "(x) => sqrt(x)", "YearsExperience")
    .Append(context.Transforms.Expression("TeamsManagedOutput", "(x, y) => x ? y : 0", nameof(SalaryInput.IsManager), nameof(SalaryInput.NumberOfTeamsManaged)))
    .Append(context.Transforms.Expression("ToLowerOutput", "(x) => lower(x)", nameof(SalaryInput.Title)));

var expressionsTransformed = expressions.Fit(data).Transform(data);

var expressionsData = context.Data.CreateEnumerable<ExpressionOutput>(expressionsTransformed,
    reuseRowObject: false);

foreach (var expression in expressionsData)
{
    Console.WriteLine($"Square Root - {expression.SquareRootOutput}");
    Console.WriteLine($"Teams Managed - {expression.TeamsManagedOutput}");
    Console.WriteLine($"To Lower - {expression.ToLowerOutput}");
    Console.WriteLine(Environment.NewLine);
}

Console.ReadLine();

Square Root - 1
Teams Managed - 0
To Lower - developer


Square Root - 3
Teams Managed - 2
To Lower - director


Square Root - 2
Teams Managed - 0
To Lower - analyst


